# SMD Python Hands On

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

## Inhalt
<div id="toc"></div>

## Grundlagen

Werden heute hier nicht besprochen, gute Einstiegspunkte sind:

* [PeP et al. Toolbox Workshop](https://toolbox.pep-dortmund.org)
* [The scientific Python lectures](https://github.com/jrjohansson/scientific-python-lectures)
* [Scientific Python Notebooks](https://github.com/maxnoe/scientific_python_notebooks)
* [A Byte Of Python](https://python.swaroopch.com/)

Insbesondere für den Datamining-Teil der Vorlesung:
* [The Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/)



## Style Guide und Linter

* Für python gibt es einen allgemein anerkannten Style Guide, pep8. Beachten!  
  `flake8` prüft gleichzeitig das Programm auf Syntax-Fehler und auf Beachtung des styleguides.
    * `conda install flake8`
    * `flake8 test.py`
* Autoformatter
    * `black` formatiert automatische Dateien so, dass sie den Style Guide befolgen
    
* Alle vernünftigen Editoren können Fehler und Nichtbefolgung eines Style Guides markieren (Linting).
  * VS Code Plugin: https://marketplace.visualstudio.com/items?itemName=ms-python.python
  * Atom-Plugin: https://atom.io/packages/linter-flake8
  * Vim-Plugins: https://github.com/neoclide/coc.nvim oder https://github.com/neomake/neomake oder https://github.com/vim-syntastic/syntastic


## conda environments

* Verschiedene Projekte können unterschiedliche und inkompatible Abhängigkeiten haben
* Guter Stil: Isolation in Umgebungen
* Für conda: conda environments
* Für SMD haben wir eine vorgefertigt, mit allen benötigten / für die Lösung der Übungen erlaubten Paketen.
* Download im Moodle: `environment.yml`.

Neue Umgebung erzeugen mithilfe der Datei:

```
$ conda env create -f environment.yml
```

Die Umgebung nutzen / aktivieren:

```
$ conda activate smd
```

Python / ipython / jupyter sollten jetzt aus dieser Umgebung kommen:
```
$ which python
/home/maxnoe/.local/anaconda3/envs/smd/bin/python
```

## numpy

Wiederholdung der wichtigsten numpy Eigenschaften.

* Python ist eine dynamische Sprache mit vielen Freiheiten.  
  Das hat viele Vorteile aber auch Trade-Offs, gerade wenn es um
  die Geschwindigkeit von numerischen Berechnungen geht

* Numpy ist schnell durch Vektorisierung und kompilierten C++/C/Cython/Fortran-Code  
   ⇒ Gute Daumenregel: Keine Python Schleifen über numpy arrays
* Viele Funktionen für Datenanalyse, Zufallszahlen, Numerik, Lineare Algebra etc
* Das "naiver" python code langsam ist, ist einer der Hauptkritikpunkte an der Sprache, trotzdem hat sie sich für Datenanalyse-Anwendungen weitgehend durchgesetzt
* Verschiedene Ansätze um python schneller zu machen (e.g. numba, cython, ...) oder zu ersetzen (julia)

In [3]:
import numpy as np

## Datentypen

Numpy unterstützt eine Vielzahl von Datentypen.
Für die Vorlesung/Übung reichen meist 3:

* bool: True / False
* int64: 64-Bit signed integer
* float64: 64-Bit floating point number 

Mehr zu Datentypen in den numerischen Grundlagen

In [4]:
# numpy arrays from python lists
floats = np.array([1.0, 3.14, 1e3])
ints = np.array([1, 2, 3])
bools = np.array([True, False, True])

print(floats.dtype, ints.dtype, bools.dtype)

float64 int64 bool


### Grundlegende Eigenschaften von np.ndarrays

In [5]:
array2d = np.array([
    [1, 2, 3],
    [4, 5, 6]
])

def array_info(a):
    print(f'len={len(a)}, size={a.size}, ndim={a.ndim}, shape={a.shape}, dtype={a.dtype}')

array_info(floats)
array_info(array2d)

len=3, size=3, ndim=1, shape=(3,), dtype=float64
len=2, size=6, ndim=2, shape=(2, 3), dtype=int64


### Indizierung & Masken

Indizierung über Slices und/oder boolean Masken

In [6]:
a = np.array([1.0, -3.5, 42, -5])
a

array([ 1. , -3.5, 42. , -5. ])

In [7]:
a[0], a[-1], a[1:-1], a[::2]

(1.0, -5.0, array([-3.5, 42. ]), array([ 1., 42.]))

In [8]:
a > 0

array([ True, False,  True, False])

In [9]:
a[a > 0]

array([ 1., 42.])

In [10]:
# parentheses are needed here
# | = or
# & = and
# ~ = not

a[(a > -10) & (a < 10)]

array([ 1. , -3.5, -5. ])

### Das axis keyword

Wichtig für aggregierende Operationen (z.B. `np.sum`, `np.mean`, `np.prod`, `np.std`)

In [11]:
X = np.arange(12).reshape(4, 3)
X

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [12]:
np.sum(X)

66

In [13]:
np.sum(X, axis=0)

array([18, 22, 26])

### Broadcasting

Docs: https://numpy.org/doc/stable/user/basics.broadcasting.html

Mit "broadcasting" kann numpy auch Arrays verschiedener Größe miteinander verrechnen, wenn diese kompatible Dimensionalität haben.

* Die letzten überlappenden Dimensionen müssen kompatibel sein.

* Dimensionen sind kompatibel wenn:

    * sie identisch sind
    * eine von ihnen 1 ist


Beispiel: shape (3, 2, 2) ist kompatibel mit shape (2, 2) weil die letzten Dimensionen identisch sind.

In [15]:
a = np.arange(12).reshape(4, 3)
b = 5
c = np.arange(3)
d = np.arange(4)

In [16]:
a

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [17]:
a.shape, c.shape, d.shape

((4, 3), (3,), (4,))

In [18]:
a - b

array([[-5, -4, -3],
       [-2, -1,  0],
       [ 1,  2,  3],
       [ 4,  5,  6]])

In [19]:
a - c

array([[0, 0, 0],
       [3, 3, 3],
       [6, 6, 6],
       [9, 9, 9]])

In [21]:
# a - d -> error
(a.T - d).T


array([[0, 1, 2],
       [2, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

In [22]:
#alternativ
a - d[:,np.newaxis]

array([[0, 1, 2],
       [2, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

### Timing Beispiel: Loops vs. Numpy vektorisiert

Aufgabe: Rechne den Punkt mit dem kleinsten Abstand zu einem anderen Punkt aus

In [24]:
point = (0, 1)
points = [
    (0, 0),
    (0.5, -0.5),
    (1, -1),
    (0, 2),
    (0, 1.1),
    (-2, 3),
    (5, 1),
    (10, 4),
    (-4, 2),
    (-3, 0),
] * 1000

Python Lösung mit for-Schleife:

In [25]:
import math

def distance(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def find_closest(points, point):
    min_distance = math.inf
    
    for i, other in enumerate(points):
        d = distance(point, other) 
        
        if d < min_distance:
            min_distance = d
            min_idx = i
    
    return min_idx

idx = find_closest(points, point)
print(idx, points[idx])

4 (0, 1.1)


In [26]:
%%timeit 
find_closest(points, point)

5.37 ms ± 25.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
points = np.array(points)
point = np.array(point)

In [28]:
def find_closest_numpy(points, point):
    distances = np.linalg.norm(points - point, axis=1)
    idx = np.argmin(distances)
    return idx

idx = find_closest_numpy(points, point)
print(idx, points[idx])

4 [0.  1.1]


In [29]:
%%timeit 
find_closest_numpy(points, point)

192 µs ± 770 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Pseudozufallszahlen

Docs: https://numpy.org/doc/stable/reference/random/index.html

In [ ]:
from numpy.random import default_rng

# create a new random generator with a fixed seed
# this avoids "evil" global state when using `np.random.seed` or `np.random.<function>`
rg = default_rng(42)

#### Eindimensionale Verteilungen

In [ ]:
uniform = rg.uniform(-5, 5, 1000)
gaussian = rg.normal(0, 1, 1000)
poisson = rg.poisson(3, 1000)

#### Mehrdimensionale Normalverteilung

In [ ]:
mean = [2, 1]
cov = [[2, 1],
       [1, 4]]
gauss_2d = rg.multivariate_normal(mean, cov, 1000)

Für reproduzierbarkeit wichtig: der Startwert (seed) des Zufallszahlengenerators.

Auch wichtig bei parallemen Rechnen oder Wiederaufnahme von Simulationen. 

In [ ]:
rg = default_rng(42)
rg.normal()

## matplotlib

Die zwei wichtigsten Plot-Arten für die Datenanalyse sind Histogramme und Streudiagramme

### Histogramme

In [ ]:
import matplotlib.pyplot as plt

# for interactive plots in the notebook.
# in ipython just use %matplotlib
%matplotlib notebook

In [ ]:
# new feature of matplotlib, use this instead of `plt.tight_layout()`
plt.figure(constrained_layout=True)
plt.hist(gaussian, bins=20, range=[-5, 5])

None # just to not mess up the notebook

Bei Vergleichen ist es wichtig, das gleiche Binning zu verwenden

In [ ]:
bins = 20
limits = [-5, 5]

plt.figure(constrained_layout=True)
plt.hist(gaussian, bins=bins, range=limits, histtype='step', label='Gaussian', lw=2)
plt.hist(uniform, bins=bins, range=limits, histtype='step', label='Uniform', lw=2)
plt.legend()


None  # just to not mess up the notebook

<span style="color: crimson; font-weight: bold; font-size: 2rem">
    Bei diskreten Werten unbedingt ganzzahlige, zentrierte bins verwenden
</span>

In [ ]:
plt.figure(constrained_layout=True)
plt.hist(poisson, bins=15)
None  # just to not mess up the notebook

In [ ]:
plt.figure(constrained_layout=True)
plt.hist(
    poisson,
    bins=np.arange(15) - 0.5,  # bins can be either number of bins or bin edges
    edgecolor='w',
    lw=2
)


np.arange(15) - 0.5

In [ ]:
# convert bin edges to bin centers and bin width

bins = np.arange(8) - 0.5

bin_centers = 0.5 * (bins[:-1] + bins[1:])
bin_widths = np.diff(bins)

print(bins)
print(bin_centers)
print(bin_widths)

### Streudiagramme

In [ ]:
plt.figure(constrained_layout=True)

plt.scatter(gauss_2d[:, 0], gauss_2d[:, 1])

In [ ]:
gauss_2d = rg.multivariate_normal(mean, cov, 10000)

plt.figure(constrained_layout=True)

# smaller dots and transparency for this many points
plt.scatter(gauss_2d[:, 0], gauss_2d[:, 1], s=5, alpha=0.2)

* Scatter kann ein Drittes Attribut als Farbe darstellen
* Mit ListedColorMap bekommt man eine diskrete Colormap mit einer Farbe für jede Klasse

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()

In [ ]:
from matplotlib.colors import ListedColormap

n_classes = len(iris.target_names)

# new ColorMap with `n_classes` discrete colors 
# using the standard color rotation C0, C1, ...
cmap = ListedColormap([f'C{i}' for i in range(n_classes)])

# automatically adjust spacing
fig, ax = plt.subplots(constrained_layout=True)

# scatter plot with colors per class
scat = ax.scatter(
    iris.data[:, 0],       # x-values, first column
    iris.data[:, 1],       # y-values, second column 
    c=iris.target,         # data to determine color
    cmap=cmap,             # colormap, converts data in c to an actual color
    vmin=-0.5,             # minimum value for the color axis
    vmax=n_classes - 0.5,  # maximum value for the color axis
)


# follow SI conventions (divide by unit)
ax.set_xlabel(iris.feature_names[0].replace('(cm)', '/ cm'))
ax.set_ylabel(iris.feature_names[1].replace('(cm)', '/ cm'))

# colorbar with ticklabels
bar = fig.colorbar(scat, ticks=[0, 1, 2], ax=ax)
bar.set_ticklabels(iris.target_names)

### 2d-Histogramme

In [ ]:
plt.figure(constrained_layout=True)
plt.hist2d(
    gauss_2d[:, 0],
    gauss_2d[:, 1],
    bins=[21, 21],
    range=[[-3, 7], [-7, 9]],
    cmap='inferno', # has more contrast than the default viridis
)
plt.colorbar()
None

Manchmal hilfreich: Logarithmische Skala auf der Farbachse

z.B. um eine zweite, kleine Population zu erkennen

In [ ]:
gauss_2d = rg.multivariate_normal(mean, cov, 100000)
gauss_2d_2 = rg.multivariate_normal([-1.5, 4.5], [[0.5, 0], [0, 0.5]], 500)

gauss_2d_both = np.concatenate([gauss_2d, gauss_2d_2], axis=0)

gauss_2d_both.shape

In [ ]:
plt.figure(constrained_layout=True)
plt.hist2d(
    gauss_2d_both[:, 0],
    gauss_2d_both[:, 1],
    bins=[50, 50],
    range=[[-3, 7], [-7, 9]],
    cmap='inferno'
)
plt.colorbar()

In [ ]:
from matplotlib.colors import LogNorm

plt.figure(constrained_layout=True)
plt.hist2d(
    gauss_2d_both[:, 0],
    gauss_2d_both[:, 1],
    bins=[50, 50],
    range=[[-3, 7], [-7, 9]],
    norm=LogNorm(),
    cmap='inferno',
)
plt.colorbar()

Oft wichtig: gleiches Seiteverhältnis der Daten:

In [ ]:
from matplotlib.colors import LogNorm

fig, ax = plt.subplots(constrained_layout=True)

ax.set_aspect('equal')

hist, xbins, ybins, plot = ax.hist2d(
    gauss_2d_both[:, 0],
    gauss_2d_both[:, 1],
    bins=[50, 50],
    range=[[-3, 7], [-7, 9]],
    norm=LogNorm(),
    cmap='inferno',
)

fig.colorbar(plot, ax=ax)

None

## scipy

Für SMD-A wird vor allem das `scipy.stats` Modul wichtig werden

### scipy.stats

Viele Statistische Verteilungen mit vielen Eigenschaften

Docs: https://docs.scipy.org/doc/scipy/reference/stats.html

In [ ]:
from scipy.stats import norm

mean = 5
std = 2

gaussian = norm(mean, std)

In [ ]:
# draw random samples
samples = gaussian.rvs(size=1000)

x = np.linspace(mean - 5 * std, mean + 5 * std, 250)

plt.figure(constrained_layout=True)

# plot histogram
plt.hist(
    samples,
    bins=100,
    range=[x.min(), x.max()],
    density=True,  # normalize histgram to an area of 1
    label='Normiertes Histogramm',
)

# plot pdf and cdf
plt.plot(x, gaussian.pdf(x), label='Wahrscheinlichkeitsdichte', lw=2)
plt.plot(x, gaussian.cdf(x), label='Verteilungsfunktion', lw=2)


plt.legend()

In [ ]:
x = rg.normal(5, 2, 100)

# maximum likelihood fit 
mean_fit, std_fit = norm.fit(x)

mean_fit, std_fit, mean, std

## Pandas

Bibliothek für Tabellarische Daten

In [ ]:
import pandas as pd

### Create a pandas.DataFrame from numpy arrays

In [ ]:
signal = pd.DataFrame({
    'x': rg.normal(2, 0.5, 1000),
    'y': rg.uniform(1, 0.5, 1000),
    'N': rg.poisson(50, 1000),
    't': rg.exponential(5, 1000),
})

### Erster Blick in die Daten

In [ ]:
signal.head()

### Input/Output

In [ ]:
signal.to_csv('data.csv')

In [ ]:
signal = pd.read_csv('data.csv', index_col=0)

In [ ]:
# HDF5 is a fast, binary data format better suited for large datasets
signal.to_hdf('data.hdf5', key='signal')

In [ ]:
background = pd.DataFrame({
    'x': rg.uniform(-4, 4, 10000),
    'y': rg.uniform(-4, 4, 10000),
    'N': rg.poisson(30, 10000),
    't': rg.exponential(10, 10000),
})

In [ ]:
# you can store more than one dataset in the same file
background.to_hdf('data.hdf5', key='background')

In [ ]:
background.head()

In [ ]:
df = pd.read_hdf('data.hdf5', key='signal')

In [ ]:
len(df)

Look at the first / last values

In [ ]:
df.head()

In [ ]:
df.describe()

### The titanic dataset

In [ ]:
df = pd.read_csv('titanic.csv')

In [ ]:
df.head()

Wie viele Valide Werte gibt es in jeder Spalte?

In [ ]:
df.count()

Spalten loswerden, die zu viele missing values haben

In [ ]:
# axis=1 Spalten droppen
# inplace=df direkt bearbeiten
df.drop(['cabin', 'boat', 'body', 'home.dest'], axis=1, inplace=True) 

df.head()

Wie war die Geschlechter Verteilung auf der Titanic?

In [ ]:
df.sex.value_counts()

Mächtige Operation: GroupBy → Aggregate

Datensatz in mehrere Gruppen unterteilen und pro Gruppe zusammenfassen.

Hier: Aufgeschlüsselt nach Geschlecht, den Prozentsatz der überlebenden.

In [ ]:
df.groupby('sex')['survived'].agg('mean')

Auch DataFrames unterstützen masken:

In [ ]:
df['child'] = df.age < 9

df[df['child']]

Alternativ

In [ ]:
df.groupby('child').survived.mean()

## Klassen


Dieses Jahr wird es eine Semesterübergreifende Aufgabe geben,
die an einer Detektor-Simulation und Analyse entlang führt.

Teilaufgaben werden als normale Übungsaufgaben gestellt und wir geben 
einen großteil der Struktur und des Codes vor.

Die Aufgaben werden daraus bestehen, den Code an bestimmten stellen zu ergänzen.

Als Vorbereitung geben wir hier eine (sehr) kurze Einführung in Python-Klassen.

Die volle Dokumentation gibt es hier:  

Docs: https://docs.python.org/3/reference/datamodel.html

In [ ]:
class Detector:
    
    # special method to initialize a new instance of a class
    def __init__(self, width, height):
        self.width = width
        self.height = height

In [ ]:
detector = Detector(width=10, height=5)

print(detector.width, detector.height)

Eine Aufgabe könnte zum Beispiel daraus bestehen,
eine Methode zu schreiben, die überprüft, ob ein Punkt innerhalb des Detektors ist.

Das Koordinatensystem sei so definiert, das width die Ausdehnung in x, height die Ausdehnung in y ist, und die untere, linke Ecke des Detektors im Ursprung.

Vorgeben könnte dann folgendes sein:

In [ ]:
class Detector:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        
    def is_inside(self, x, y):
        result = False
        
        ##########
        # Hier Code ergänzen, um korrektes Ergebnis zu berechnen
        ##########
        
        return result

Wir werden meistens Skripte oder *Tests* zur Verfügung stellen,
mit denen ihr eure Ergebnis überprüfen könnt:

Docs: https://docs.pytest.org/en/stable/

In [ ]:
def test_is_inside():
    # a test function to be used with `pytest`
    d = Detector(5, 5)

    assert d.is_inside(2, 2), 'This point should be inside'
    assert not d.is_inside(10, 10), 'This point should not be inside'
    
test_is_inside()

In Python, werden Operatoren und vieles weitere durch sogenannte "Magic Methods"
implementiert, die mit zwei Unterstrichen ("dunder") beginnen:

In [ ]:
print(Detector(10, 5))

In [ ]:
class Detector:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def __repr__(self):
        '''This methods enables conversion to a representation str, e.g. for printing'''
        return f'{self.__class__.__name__}(width={self.width}, height={self.height})'

In [ ]:
d = Detector(10, 5)
print(d)